In [62]:
import pandas as pd
import numpy as np
import random
from copy import deepcopy as copy

In [63]:
dice_values = [(4, 1), (6, 2), (8, 3), (10, 4), (10, 4), (12, 5), (20, 6)]
goal = 20
player_1_total = 0
player_2_total = 0

In [64]:
player_1_money = 100
player_2_money = 100

In [65]:
data = pd.DataFrame(columns=["round", "player 1 strategy", 
                             "player 2 strategy", 
                             "player 1 result", 
                             "player 2 result", 
                             "player 1 money", 
                             "player 2 money"])

In [66]:
class Player:
    def __init__(self, name, strategy, money, active):
        self.name = name
        self.strategy = strategy
        self.money = money
        self.active = active
        self.total = 0
    def __repr__(self):
        return self.name
class Die:
    def __init__(self, value, cost):
        self.value = value
        self.cost = cost
    def __repr__(self):
        return str(self.value)
original_dice_list = [Die(x, y) for x, y in dice_values]
original_dice_list

[4, 6, 8, 10, 10, 12, 20]

In [67]:
def StartHighNeverBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighSometimesBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if ((y.value*2)/3)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighFrequentlyBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if (y.value/2)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartLowStrategy(total, goal, money, dice):
    dice.sort(key=lambda x: x.value)
    print(dice)
    gap = goal - total
    print(gap)
    if gap<0:
        raise Exception
    for y in dice:
        print(y)
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

        

In [68]:
def RunGame(player_1, player_2, dice_list, results_df, goal = 20, blind = 1, rounds=None):
    player_1 = copy(player_1)
    player_2 = copy(player_2)
    pot = 0
    round_number = 1
    print(dice_list)
    dice_list = copy(dice_list)
    initial_dice_list = copy(dice_list)
    if random.random()<0.5:
        player_order = [player_1, player_2]
    else:
        player_order = [player_2, player_1]
    player_1.active = True
    player_2.active = True
    while (rounds is None or round_number<=rounds) and (player_1.money>(10*blind) and player_2.money>(10*blind)):
        player_1.money = player_1.money - blind
        pot = pot + blind
        player_2.money = player_2.money - blind
        pot = pot + blind
        while len(dice_list)>0 and (player_1.active is True or player_2.active is True):
            for z in player_order:
                if z.active is True:
                    try:
                        die = z.strategy(z.total, goal, z.money, dice_list)
                        print("Die is {}".format(die))
                    except Exception as e:
                        z.active = False
                        z.total = 0
                        die = "Hold"
                        print("Player busted")
                        print(e)
                    if die != "Hold":
                        z.total = z.total + (random.randint(1, die.value))
                        if z.total>goal:
                            z.active = False
                            print("{} busted".format(z.name))
                        z.money = z.money - die.cost
                        pot = pot + die.cost
                        print(pot)
                        dice_list.remove(die)
                        print("Removing {} from dice list".format(die))
                        dice_list
                    else:
                        z.active = False
        if player_1.total> goal:
            player_1.total = 0
        if player_2.total > goal:
            player_2.total = 0
        if player_1.total == player_2.total:
            player_1.money = player_1.money + (pot/2)
            player_2.money = player_2.money + (pot/2)
            pot = 0
        if player_1.total > player_2.total:
            player_1.money = player_1.money + pot
            pot = 0
        if player_2.total > player_1.total:
            player_2.money = player_2.money + pot
            pot = 0
        results_dict = {"round": str(round_number), 
                        "player 1 strategy": player_1.strategy, 
                            "player 2 strategy": player_2.strategy, 
                            "player 1 result": player_1.total, 
                            "player 2 result": player_2.total, 
                            "player 1 money": player_1.money, 
                            "player 2 money": player_2.money}
        print(results_dict)
        results_df = results_df.append(results_dict, ignore_index=True)
        round_number = round_number + 1
        print("Round Number is {}".format(round_number))
        dice_list = copy(initial_dice_list)
        print(dice_list)
        player_1.total = 0
        player_2.total = 0
        player_1.active = True
        player_2.active = True
        player_order = player_order[::-1]
    return results_df


        

In [69]:
start_high_never_bust = Player("Player 1", StartHighNeverBustStrategy, player_1_money, True)
start_low = Player("Player 2", StartLowStrategy, player_2_money, True)
start_high_sometimes_bust = Player("Player 3", StartHighSometimesBustStrategy, player_1_money, True)
start_high_frequently_bust = Player("Player 4", StartHighFrequentlyBustStrategy, player_2_money, True)

In [70]:
start_low.strategy(0, 20, 100, original_dice_list)

[4, 6, 8, 10, 10, 12, 20]
20
4


4

In [71]:
results_1 = RunGame(start_high_never_bust, start_low, original_dice_list, data)
results_1

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 12
14
Removing 12 from dice list
[6, 8, 10, 10]
18
6
Die is 6
16
Removing 6 from dice list
[10, 10, 8]
Die is 10
20
Removing 10 from dice list
[8, 10]
12
8
Die is 8
23
Removing 8 from dice list
[10]
Die is Hold
[10]
10
10
Die is 10
27
Removing 10 from dice list
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd6710c5048>, 'player 2 strategy': <function StartLowStrategy at 0x7fd67112c268>, 'player 1 result': 14, 'player 2 result': 12, 'player 1 money': 111, 'player 2 money': 89}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
16
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is Hold
[8, 10, 10, 12]
11
8
Die is 8
14
Removing

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,14,12,111,89
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,20,12,118,82
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,13,12,129,71
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,13,12,140,60
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,17,17,142,58
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,17,13,158,42
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,17,16,169,31
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,16,13,176,24
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,18,11,187,13
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd67112c268>,13,13,186.5,13.5


In [72]:
results_2 = RunGame(start_high_never_bust, start_high_sometimes_bust, original_dice_list, data)
results_2

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is Hold
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 from dice list
[6, 4]
Die is 4
25
Removing 4 from dice list
[6]
Die is Hold
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd6710c5048>, 'player 2 strategy': <function StartHighSometimesBustStrategy at 0x7fd6608cc488>, 'player 1 result': 17, 'player 2 result': 18, 'player 1 money': 82, 'player 2 money': 118}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 4
14
Removing 4 from dice list
[10, 10, 8, 6]
Die is 10
Player 3 busted
18
Removing 10 from dice list
[10, 8, 6]
Die is Hold
{'round':

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,18,82,118
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,0,92,108
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,12,18,77,123
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,18,70,130
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,17,71.5,128.5
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,18,19,64.5,135.5
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,18,20,46.5,153.5
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,19,15,60.5,139.5
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,20,50.5,149.5
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,15,15,49,151


In [73]:
results_3 = RunGame(start_high_never_bust, start_high_frequently_bust, original_dice_list, data)
results_3

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 8
20
Removing 8 from dice list
[10, 6, 4]
Die is 10
24
Removing 10 from dice list
[6, 4]
Die is Hold
[6, 4]
Die is 4
25
Removing 4 from dice list
[6]
Die is Hold
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd6710c5048>, 'player 2 strategy': <function StartHighFrequentlyBustStrategy at 0x7fd6608ccae8>, 'player 1 result': 18, 'player 2 result': 19, 'player 1 money': 91, 'player 2 money': 109}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 4
22
Removing 4 from dice list
[8, 6]
D

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,19,91,109
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,17,19,79,121
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,19,19,76.5,123.5
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,19,19,79,121
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,0,91,109
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,19,83,117
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,12,20,69,131
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,15,10,81,119
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,19,19,83.5,116.5
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,13,11,95.5,104.5


In [74]:
results_4 = RunGame(start_low, start_high_sometimes_bust, original_dice_list, data)
results_4

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 8
Player 3 busted
12
Removing 8 from dice list
[6, 10, 10, 12]
17
6
Die is 6
14
Removing 6 from dice list
[10, 10, 12]
11
10
Die is 10
18
Removing 10 from dice list
[10, 12]
10
10
Die is 10
22
Removing 10 from dice list
[12]
0
12
Die is Hold
{'round': '1', 'player 1 strategy': <function StartLowStrategy at 0x7fd67112c268>, 'player 2 strategy': <function StartHighSometimesBustStrategy at 0x7fd6608cc488>, 'player 1 result': 20, 'player 2 result': 0, 'player 1 money': 110, 'player 2 money': 90}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
18
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is 12
Player 3 busted
16
Removing 12 from dice list
[8, 10, 10]
17

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,20,0,110,90
1,2,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,16,0,122,78
2,3,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,12,16,115,85
3,4,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,13,18,104,96
4,5,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,11,18,95,105
5,6,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,13,19,88,112
6,7,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,8,18,81,119
7,8,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,18,0,97,103
8,9,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,17,16,113,87
9,10,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighSometimesBustStrategy at 0x...,11,18,106,94


In [75]:
results_5 = RunGame(start_low, start_high_frequently_bust, original_dice_list, data)
results_5

[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
16
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is 12
16
Removing 12 from dice list
[8, 10, 10]
15
8
Die is 8
19
Removing 8 from dice list
[10, 10]
Die is Hold
[10, 10]
10
10
Die is 10
23
Removing 10 from dice list
[10]
1
10
Die is Hold
{'round': '1', 'player 1 strategy': <function StartLowStrategy at 0x7fd67112c268>, 'player 2 strategy': <function StartHighFrequentlyBustStrategy at 0x7fd6608ccae8>, 'player 1 result': 19, 'player 2 result': 19, 'player 1 money': 100.5, 'player 2 money': 99.5}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 12
14
Removing 12 from dice list
[6, 8, 10, 10]
18
6
Die is 6
16
Removing 6 from dice list
[10, 10, 8]
Die is 10


,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,19,19,100.5,99.5
1,2,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,16,17,89.5,110.5
2,3,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,16,20,78.5,121.5
3,4,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,15,20,71.5,128.5
4,5,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,11,19,64.5,135.5
5,6,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,13,0,76.5,123.5
6,7,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,15,13,96.5,103.5
7,8,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,14,19,89.5,110.5
8,9,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,19,16,105.5,94.5
9,10,<function StartLowStrategy at 0x7fd67112c268>,<function StartHighFrequentlyBustStrategy at 0...,13,20,98.5,101.5
